In [1]:
import xarray as xr
import cv2
import os
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from matplotlib.backends.backend_pdf import PdfPages
from matplotlib.animation import FFMpegWriter
from tqdm import tqdm

In [2]:
pooled = xr.open_dataset(r'Z:\jumping\reform_clipped_vids_DLC_params\pooled_jump_data.nc')

In [7]:
config = {'data_path':'/Users/Niell Lab/Desktop/'}

In [27]:
# make plots using the pooled jumping data
def pooled_jump_analysis(pooled, config):

    pdf = PdfPages(os.path.join(config['data_path'], 'pooled_jump_plots7.pdf'))
    
    # convert to dataarray so that indexing can be done accross recordings
    # this is only needed if there's more than one trial read in, so a try/except is used
    try:
        pooled_da = pooled.to_array()
    except AttributeError:
        pooled_da = pooled
    # then, get data out for each parameter
    all_pitch = pooled_da.sel(jump_params='head_pitch').values
    all_phi = pooled_da.sel(jump_params='mean_eye_phi').values
    all_div = pooled_da.sel(jump_params='eye_th_div').values
    all_th_gaze = pooled_da.sel(jump_params='th_gaze', frame=range(60)).values
    all_th_div = pooled_da.sel(jump_params='th_div', frame=range(60)).values
    all_th_phi = pooled_da.sel(jump_params='th_phi', frame=range(60)).values
    lags = range(-30, 30)
    
    dwnspl = 100
    
    # head theta, phi
    plt.figure(figsize=(5,5))
    plt.plot(all_pitch[::dwnspl], all_phi[::dwnspl], 'k.')
    plt.xlabel('head pitch'); plt.ylabel('phi')
    plt.xlim([-60,60]); plt.ylim([-30,30])
    plt.plot([-60,60],[-60,60],':',color=[0.5,0.5,0.5])
    pdf.savefig()
    plt.close()
    # head theta, eye theta divergence
    plt.figure(figsize=(5,5))
    plt.plot(all_pitch[::dwnspl], all_div[::dwnspl], 'k.')
    plt.xlabel('head pitch'); plt.ylabel('eye theta div')
    plt.xlim([-60,60]); plt.ylim([-30,30])
    plt.plot([-60,60],[60,-60],':',color=[0.5,0.5,0.5])
    pdf.savefig()
    plt.close()
    # xcorr with head angle
    plt.figure(figsize=(5,5))
    plt.errorbar(lags, np.mean(all_th_gaze,0), yerr=(np.std(np.array(all_th_gaze,dtype=np.float64),0)/np.sqrt(np.size(all_th_gaze,0))))
    plt.errorbar(lags, np.mean(all_th_div,0), yerr=(np.std(np.array(all_th_div,dtype=np.float64),0)/np.sqrt(np.size(all_th_div,0))))
    plt.errorbar(lags, np.mean(all_th_phi,0), yerr=(np.std(np.array(all_th_phi,dtype=np.float64),0)/np.sqrt(np.size(all_th_phi,0))))
    plt.ylim([-1,1]); plt.ylabel('correlation'); plt.title('xcorr with head pitch')
    plt.legend(['mean theta', 'theta divergence', 'mean phi'])
    pdf.savefig()
    plt.close()

    pdf.close()

In [28]:
pooled_jump_analysis(pooled, config)